In [1]:
import pyranges as pr
import pandas as pd
import numpy as np
import h5py 
import os

from cerberus.cerberus import *
from cerberus.main import *


## annotate transcriptome

## wrapper

In [2]:
def check_files(files, sources):
    """
    Check if a series of files exists and whether their 
    associated sources are all unique. Raise an exception
    if not.
    
    Parameters:
        files (list of str): List of file paths 
        sources (list of str): List of sources
    """
    
    for f in files:
        if not os.path.exists(f):
            raise Exception('File {} does not exist'.format(bed))

    if len(set(sources)) < len(sources):
        raise Exception('Sources must be unique')

In [20]:
# def gen_reference(ref_gtf, o, ref_tss, ref_tes,
#                   gtf_tss_dist, gtf_tss_slack,
#                   gtf_tes_dist, gtf_tes_slack,
#                   tss_slack, tes_slack,
#                   verbosity, tmp_dir,
#                   keep_tmp):
    
#     # parse config files and check if the input files exist
#     gtfs, gtf_add_ends, gtf_sources = parse_gtf_config(ref_gtf)
#     tss_beds, tss_add_ends, tss_sources = parse_agg_ends_config(ref_tss)
#     tes_beds, tes_add_ends, tes_sources = parse_agg_ends_config(ref_tes)
    
#     check_files(gtfs+tss_beds, gtf_sources+tss_sources)
#     check_files(gtfs+tes_beds, gtf_sources+tss_sources)
    
#     # get ends / ics from gtfs
#     gtf_tss_beds = []
#     gtf_tes_beds = []
#     ics = []
#     for gtf, source in zip(gtfs, gtf_sources):
        
#         if verbosity > 0:
#             print('Finding TSSs from {}'.format(source))
            
#         tss_fname = '{}/{}_tss.bed'.format(tmp_dir, source)
#         gtf_to_bed(gtf, 'tss', tss_fname, gtf_tss_dist, gtf_tss_slack)

#         if verbosity > 0:
#             print('Finding TSSs from {}'.format(source))
            
#         tes_fname = '{}/{}_tes.bed'.format(tmp_dir, source)
#         gtf_to_bed(gtf, 'tes', tes_fname, gtf_tes_dist, gtf_tes_slack) 

#         if verbosity > 0:
#             print('Finding intron chains from {}'.format(source))
        
#         ic_fname = '{}/{}_ic.tsv'.format(tmp_dir, source)
#         ic = gtf_to_ics(gtf, ic_fname)
        
#         gtf_tss_beds.append(tss_fname)
#         gtf_tes_beds.append(tes_fname)
#         ics.append(ic_fname)
    
#     # get list of tmp files to (maybe) remove later
#     tmp_files = gtf_tss_beds + gtf_tes_beds + ics
    
#     # aggregate ends
#     tss_beds = gtf_tss_beds + tss_beds
#     tss_add_ends = gtf_add_ends + tss_add_ends
#     tss_sources = gtf_sources + tss_sources
    
#     if verbosity > 0:
#         print('Aggregating TSSs')
    
#     tss = aggregate_ends(tss_beds, 
#                          tss_sources,
#                          tss_add_ends, 
#                          tss_slack,
#                          'tss')
    
#     tes_beds = gtf_tes_beds + tes_beds
#     tes_add_ends = gtf_add_ends + tes_add_ends
#     tes_sources = gtf_sources + tes_sources
    
#     if verbosity > 0:
#         print('Aggregating TESs')
    
#     tes = aggregate_ends(tes_beds,
#                          tes_sources, 
#                          tes_add_ends,
#                          tes_slack,
#                          'tes')
    
#     # aggregate ics
#     if verbosity > 0:
#         print('Aggregating intron chains')
        
#     ics = aggregate_ics(ics, gtf_sources)
    
#     # write h5df
#     write_h5(ics, tss, tes, o)
    
#     # delete tmp files
#     if not keep_tmp:
#         for f in tmp_files:
#             os.remove(f)
            

In [2]:
ref_gtf = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/gtf_config.csv'
ref_tss = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/tss_no_gtf.csv'
ref_tes = None
gtf_tss_slack = 50
gtf_tss_dist = 50
gtf_tes_dist = 50
gtf_tes_slack = 50
tss_slack = 20
tes_slack = 20
tmp_dir = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/'
o = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/bepis.h5'

gen_reference(ref_gtf, ref_tss, ref_tes,
                  gtf_tss_dist, gtf_tss_slack,
                  gtf_tes_dist, gtf_tes_slack,
                  tss_slack, tes_slack,
                  tmp_dir,
                  o)

In [3]:
ic, tss, tes, _ = read_h5(o)

In [4]:
print(tss.head())
print(tss.source.unique())
print(tes.head())
print(tes.source.unique())
print(ic.head())
print(ic.source.unique())

+--------------+-----------+-----------+--------------+---------------------+------------+
| Chromosome   |     Start |       End | Strand       | Name                | source     |
| (category)   |   (int32) |   (int32) | (category)   | (object)            | (object)   |
|--------------+-----------+-----------+--------------+---------------------+------------|
| SIRV1        |     10597 |     10698 | +            | ENCODEHG000058846_1 | talon      |
| SIRV1        |     10315 |     10416 | -            | ENCODEHG000058837_1 | talon      |
| SIRV1        |     10739 |     10840 | -            | ENCODEHG000058837_2 | talon      |
| SIRV2        |      4713 |      4814 | -            | ENCODEHG000058784_1 | talon      |
| SIRV2        |      5855 |      5956 | -            | ENCODEHG000058784_2 | talon      |
| SIRV3        |      1930 |      2031 | +            | ENCODEHG000058844_1 | talon      |
| SIRV3        |      3953 |      4054 | +            | ENCODEHG000058844_2 | talon      |

  Chromosome Strand                                        Coordinates  \
0       chr1      +  100038316-100049908-100050004-100058665-100058...   
1       chr1      +            100058728-100059877-100060005-100061834   
2       chr1      +                                          100099490   
3       chr1      +                    1001263-1008193-1008279-1013983   
4       chr1      +                    1001281-1008193-1008279-1013983   

                Name         source  
0  ENSG00000156875_1  v39,v29,talon  
1  ENSG00000156875_2        v39,v29  
2  ENSG00000226952_1  v39,v29,talon  
3  ENSG00000187608_3  v39,v29,talon  
4  ENSG00000187608_2  v39,v29,talon  
  Chromosome  Start    End Strand                 Name source
1      SIRV1  10315  10416      -  ENCODEHG000058837_1  talon
0      SIRV1  10597  10698      +  ENCODEHG000058846_1  talon
2      SIRV1  10739  10840      -  ENCODEHG000058837_2  talon
3      SIRV2   4713   4814      -  ENCODEHG000058784_1  talon
4      SIRV2   58

In [6]:
# gtfs
v39_gtf = '/Users/fairliereese/mortazavi_lab/ref/gencode.v39/gencode.v39.annotation.gtf'
v29_gtf = '/Users/fairliereese/mortazavi_lab/ref/gencode.v29/gencode.v29.annotation.gtf'
talon_gtf = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/talon/human_known_nic_nnc_talon.gtf'

gtfs = [v39_gtf, v29_gtf, talon_gtf]
add_ends = [True, True, True]
sources = ['v39', 'v29', 'talon']

fname = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/gtf_config.csv'
df = pd.DataFrame()

df['fname'] = gtfs
df['add_ends'] = add_ends
df['source'] = sources

df.to_csv(fname, sep=',', header=None, index=False)

In [6]:
# tss 
ccre_bed = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/test_no_gid_no_strand.bed'
sources = ['ccre']
add_ends = [True]

fname = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/tss_no_gtf.csv'
df = pd.DataFrame()

df['fname'] = [ccre_bed]
df['add_ends'] = add_ends
df['source'] = sources

df.to_csv(fname, sep=',', header=None, index=False)